In [2]:
# import Library
import os, cv2, numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
import pickle  # for save model

# path dataset
DATASET_PATH = "dataset/"

# label for everyone
labels = []
faces = []
label_map = {}  # save index -> name

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def extract_face(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    faces_detected = face_cascade.detectMultiScale(img, scaleFactor=1.2, minNeighbors=5)
    
    for (x, y, w, h) in faces_detected:
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (100, 100))  # face size normalization
        return face.flatten()  # change to 1D array
    return None

# process dataset
label_id = 0
for person in os.listdir(DATASET_PATH):
    person_path = os.path.join(DATASET_PATH, person)
    if os.path.isdir(person_path):
        label_map[label_id] = person  # save index to name
        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            face_data = extract_face(image_path)
            if face_data is not None:
                faces.append(face_data)
                labels.append(label_id)
        label_id += 1

# convert to numpy array
faces = np.array(faces)
labels = np.array(labels)

# dimension reduction with PCA
pca = PCA(n_components=10).fit(faces)
faces_pca = pca.transform(faces)

# train model KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(faces_pca, labels)

# save model
with open("face_recognition_model.pkl", "wb") as f:
    pickle.dump((pca, knn, label_map), f)

print("The model has been successfully trained and saved!")

The model has been successfully trained and saved!


In [3]:
# Load model
with open("face_recognition_model.pkl", "rb") as f:
    pca, knn, label_map = pickle.load(f)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# open video
video_path = "drstrange_.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Can't open the video!")
    exit()

frame_rate = cap.get(cv2.CAP_PROP_FPS)  # Frame per second
timestamps = {name: [] for name in label_map.values()}  # save time

while True:
    ret, frame = cap.read()
    if not ret:
        break  # out from loop if the video done

    frame_pos = cap.get(cv2.CAP_PROP_POS_FRAMES)  # current frame position
    time_sec = frame_pos / frame_rate  # count the time in second

    # convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # face detectiom
    faces_detected = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces_detected:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (100, 100))  # size normalization
        face_flatten = face.flatten().reshape(1, -1)

        # PCA transformation
        face_pca = pca.transform(face_flatten)

        # KNN prediction
        pred_label = knn.predict(face_pca)[0]
        person_name = label_map[pred_label]

        # save timestamp
        timestamps[person_name].append(time_sec)

        # draw a box around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, person_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Video", frame)

    # press 'q' for out from video
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# save result to file
with open("output.txt", "w") as f:
    for person, times in timestamps.items():
        f.write(f"{person} detected {len(times)} times\n")
        f.write("Appears in time:\n")
        f.write("\n".join([f"{int(t//60)} minute {int(t%60)} second" for t in times]) + "\n\n")

# show result
for person, times in timestamps.items():
    print(f"{person} detected {len(times)} times")
    print("Appears in time:")
    print("\n".join([f"{int(t//60)} minute {int(t%60)} second" for t in times]))
    print("\n")

ben detected 560 times
Appears in time:
0 minute 0 second
0 minute 0 second
0 minute 0 second
0 minute 0 second
0 minute 0 second
0 minute 0 second
0 minute 0 second
0 minute 0 second
0 minute 0 second
0 minute 0 second
0 minute 0 second
0 minute 1 second
0 minute 1 second
0 minute 1 second
0 minute 1 second
0 minute 1 second
0 minute 1 second
0 minute 1 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 2 second
0 minute 3 second
0 minute 3 second
0 minute 3 second
0 minute 3 second
0 minute 3 second
0 minute 3 second
0 minute 3 second
0 minute 3 second
0 minute 3 second
0 minute 3 second
0 minute 3 second
0 minu